In [1]:
import pipeline
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder,OrdinalEncoder,FunctionTransformer,PowerTransformer
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB

In [2]:
data=pd.read_csv(r"C:\Users\Hari D\Downloads\spam.csv",encoding='latin-1',usecols=["v1","v2"])

In [3]:
df=data.copy()
df

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [5]:
fv=df.iloc[:,1]
cv=df.iloc[:,0]

In [6]:
x_train,x_test,y_train,y_test = train_test_split(fv,cv,test_size=0.2,random_state=1,stratify= cv)

In [7]:
def lowers(x):
    return x.str.lower()

In [8]:
def html(x):
    return x.apply(lambda x:re.sub("<.+?>"," ",x))

In [9]:
def url(x):
    return x.apply(lambda x:re.sub("http[s]?://.+? +"," ",x))

In [10]:
def unw(x):
    return x.apply(lambda x:re.sub("[^a-z\s]", " ", x))

In [11]:
pre_pro_pi=Pipeline([("lower",FunctionTransformer(lowers)),
                     ("html",FunctionTransformer(html)),
                     ("url",FunctionTransformer(url)),
                    ("unw",FunctionTransformer(unw))])

In [12]:
pre_pro_pi

Pipeline(steps=[('lower',
                 FunctionTransformer(func=<function lowers at 0x000001771F93B490>)),
                ('html',
                 FunctionTransformer(func=<function html at 0x000001771F93B760>)),
                ('url',
                 FunctionTransformer(func=<function url at 0x000001771F93B520>)),
                ('unw',
                 FunctionTransformer(func=<function unw at 0x000001771FA1C160>))])

In [13]:
pre_pro_pi.fit_transform(x_train)

2357    okay same with me  well thanks for the clarifi...
5568                will    b going to esplanade fr home 
2985    reply to win       weekly  what professional s...
951                              shb b ok lor    thanx   
647     private  your      account statement for shows...
                              ...                        
4771    hi  mobile no    lt   gt   has added you in th...
1225    sir  you will receive the account no another  ...
1196    what he said is not the matter  my mind saying...
1769    ha    both of us doing e same thing  but i got...
3281    hey you around  i ve got enough for a half   t...
Name: v2, Length: 4457, dtype: object

In [14]:
final_pip = Pipeline([("pre_process",pre_pro_pi),("vectorizer",CountVectorizer())])

In [15]:
final_pip.fit_transform(x_train)

<4457x6871 sparse matrix of type '<class 'numpy.int64'>'
	with 57689 stored elements in Compressed Sparse Row format>

In [16]:
final_pip.fit_transform(x_test)

<1115x3166 sparse matrix of type '<class 'numpy.int64'>'
	with 14103 stored elements in Compressed Sparse Row format>

In [17]:
import pickle

In [18]:
mb=MultinomialNB()
model=mb.fit(final_pip.fit_transform(x_train),y_train)

In [19]:
model

MultinomialNB()

In [20]:
pickle.dump(final_pip,open(r"C:\Users\Hari D\OneDrive\Documents\visual studio\spam.pkl","wb"))
pickle.dump(model,open(r"C:\Users\Hari D\OneDrive\Documents\visual studio\model.pkl","wb"))

In [21]:
fm = pickle.load(open(r"C:\Users\Hari D\OneDrive\Documents\visual studio\spam.pkl","rb"))
model1 = pickle.load(open(r"C:\Users\Hari D\OneDrive\Documents\visual studio\model.pkl","rb"))

In [22]:
fm

Pipeline(steps=[('pre_process',
                 Pipeline(steps=[('lower',
                                  FunctionTransformer(func=<function lowers at 0x000001771F93B490>)),
                                 ('html',
                                  FunctionTransformer(func=<function html at 0x000001771F93B760>)),
                                 ('url',
                                  FunctionTransformer(func=<function url at 0x000001771F93B520>)),
                                 ('unw',
                                  FunctionTransformer(func=<function unw at 0x000001771FA1C160>))])),
                ('vectorizer', CountVectorizer())])

In [23]:
model1

MultinomialNB()

In [24]:
"""s.title('given mail is spam or ham')
q = s.text_input("enter your text")
q = final_pip.transform(pd.Series(q))
pred = model.predict(q)
if s.button("Click"):
    s.write(pred)"""

's.title(\'given mail is spam or ham\')\nq = s.text_input("enter your text")\nq = final_pip.transform(pd.Series(q))\npred = model.predict(q)\nif s.button("Click"):\n    s.write(pred)'

In [25]:
df[df.v1=='ham']["v2"].iloc[5]

"As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune"

In [26]:
df[df.v1=='spam']["v2"].iloc[5]

'URGENT! You have won a 1 week FREE membership in our å£100,000 Prize Jackpot! Txt the word: CLAIM to No: 81010 T&C www.dbuk.net LCCLTD POBOX 4403LDNW1A7RW18'